In [1]:
import sys
import os
import numpy as np
import pygad
import warnings
import time
import csv
import pandas as pd

warnings.filterwarnings('ignore')
from pathlib import Path
sys.path.insert(1, str( (Path().absolute())  ) + "/src")

from parameters import  repeat_elements, refueling_duration_estimate, fuel_cycle_length 
from schedule_mixed_reactor_optimizer import capacity_factor_weeks_approach_mix_reactors
from economic_FOMs import level_cost_of_energy_reactor_mix_starting_from_BOAK
from parameters import OM_cost_per_MWh, fuel_cycle_length, refueling_duration_estimate
from economic_FOMs import tot_TCI_multiple_reactors_mix_starting_from_BOAK




In [2]:
# The O&M cost for multiple reactors assuming

def simplified_schedule(power_list, num_of_reactors_list, demand, levelization_period_years):
    OM__cost_per_year_per_reactor_list = []
    actual_output_per_reactor_list = []

    for i in range(len(power_list)):
        p = power_list[i]
        
        # calculate the capacity factor
        CF = 1- (refueling_duration_estimate(p)/fuel_cycle_length(p))
        
        if num_of_reactors_list[i] >0:
            OM_for_specific_power = OM_cost_per_MWh(p, num_of_reactors_list[i]) # $/MWh for one reactor
        else:
            OM_for_specific_power = 0
        
        
        OM__cost_per_year_per_reactor = OM_for_specific_power *power_list[i]*num_of_reactors_list[i]*8760 * CF # dollar per year for specific power
        
        OM__cost_per_year_per_reactor_list.append(OM__cost_per_year_per_reactor)
        
        # total energy produced
        actual_output_per_reactor_per_year = p*num_of_reactors_list[i ] * 8760*CF # MWh (in a year)for specific power
        actual_output_per_reactor_list.append(actual_output_per_reactor_per_year )
       
        
    total_OM__cost_per_year = sum(OM__cost_per_year_per_reactor_list) # In dollars for all the reactors
    actual_output_per_year = sum (actual_output_per_reactor_list)
    
    
    total_OM__cost_per_year_list = [total_OM__cost_per_year] * int(levelization_period_years) # dollars per year
    actual_output_per_year_list = [actual_output_per_year] * int(levelization_period_years) # MWh
    actual_demand_per_year_list = [demand*8760] * int(levelization_period_years) # in mwh

    if actual_output_per_year >= demand*8760:
        actual_output_per_year_per_demand = demand*8760  #MWh
        excess_elec_per_year = actual_output_per_year - demand*8760
        
    elif actual_output_per_year < demand*8760: 
        excess_elec_per_year = 0
        actual_output_per_year_per_demand  = actual_output_per_year
        
    overall_capacity_factor = (actual_output_per_year/ (demand*8760))
    actual_output_per_year_per_demand_list = [actual_output_per_year_per_demand] * int(levelization_period_years)  # MWh  
    excess_elec_per_year_list = [excess_elec_per_year] * int(levelization_period_years)  

    return  total_OM__cost_per_year_list, actual_output_per_year_list, actual_output_per_year_per_demand_list , excess_elec_per_year_list, overall_capacity_factor

# example
# simplified_schedule(power_list, num_of_reactors_list, demand, levelization_period_years)[2][0]
# # simplified_schedule(power_list, num_of_reactors_list, demand, levelization_period_years)[0][0]


In [3]:
def level_cost_of_energy_reactor_mix_starting_from_BOAK_simplified( demand, interest_rate, num_of_reactors_list, power_list, levelization_period_years, elec_price):
    
    
    simplified_schedule_results = simplified_schedule(power_list, num_of_reactors_list, demand, levelization_period_years)
    list_of_OM_cost_per_year_all_reactors = simplified_schedule_results[0]
    list_of_sold_electricity_MWh_per_year_from_all_reactors= simplified_schedule_results[3]
    list_of_generated_MWh_per_year_from_all_reactors_per_demand = simplified_schedule_results[2]
    overall_capacity_factor =  simplified_schedule_results [4]
    
    
    
    sum_cost = 0 # initialization 
    sum_elec = 0 # initialization 
    for year in range( len(list_of_generated_MWh_per_year_from_all_reactors_per_demand)):
        if year == 0:
            cap_cost =  tot_TCI_multiple_reactors_mix_starting_from_BOAK(power_list, interest_rate, num_of_reactors_list)
            OM_cost_per_year = 0
            elec_gen = 0
            revenue = 0
        
        elif year > 0:
         
            cap_cost = 0 
            
            OM_cost_per_year =  list_of_OM_cost_per_year_all_reactors[year-1]
            revenue = elec_price * list_of_sold_electricity_MWh_per_year_from_all_reactors[year-1]
            elec_gen =  list_of_generated_MWh_per_year_from_all_reactors_per_demand[year-1]
        
        sum_cost += (cap_cost + OM_cost_per_year - revenue) / ((1 +interest_rate)**(year) ) 
        sum_elec += elec_gen/ ((1 + interest_rate)**year) 
    LCOE =  sum_cost/ sum_elec
    # print("LCOE NOW is: ", LCOE)
    return LCOE, overall_capacity_factor


# LCOE Optimization

In [4]:

# delete the output file is exists
filename = "GA_results_simple_case.csv"
try:
    os.remove(filename)
except OSError:
    pass


def initial_pop_reactors(power_list, sol_per_pop, possible_solutions):
    """
    This function initializes a population of potential solutions for the reactor mix optimization problem.

    Parameters:
    - power_list (list): A list of possible reactor capacities.
    - sol_per_pop (int): The number of potential solutions per population.
    - possible_solutions (list): A list of upper limits for each reactor capacity.

    Returns:
    - pop (list): A list of potential solutions, where each sub-list represents a solution and contains integers representing the number of each reactor type.
    """
    pop = np.zeros((sol_per_pop, len(power_list)))
    for i in range(len(power_list)):
        temp = np.random.randint(0, max(possible_solutions[i]) + 1, size=sol_per_pop).tolist()
        pop[:, i] = temp
    return pop.tolist()







def gene_upper_limit(power_list, demand): # limiting the maximum number of each type of reactors

    upper_limit_list = []
    for i in range(len(power_list )):
        # I use the multiplier 1.05 because of the ratio between refueling duration and operational lifetime is will nnot be higher than 1.05
        upper_limit =range(int(np.ceil(1.05*demand/power_list[i]))+2) ## TODO check the +1
        upper_limit_list.append(upper_limit )

    return  upper_limit_list













fitness_list = []
gen_sol_list = []

def on_gen(ga_instance):
    pass
    # print("Generation : ", ga_instance.generations_completed,  ga_instance.best_solution()[0], ga_instance.best_solution()[1])
    # fitness_list.append(ga_instance.best_solution()[1])
    # gen_sol_list.append(ga_instance.best_solution()[0])

def optimize_lcoe(power_list,  levelization_period_weeks, demand , interest_rate, capacity_factor_criteria, elec_price):
    solutions_list = []
    CF_list = []
    LCOE_list = []
    start_time = time.time()
    levelization_period_years = int(levelization_period_weeks/52.1428571429)
    
    # GA params
    sol_per_pop =  150 *int( (len(power_list)))
    
    num_generations = 5000
    
    num_parents_mating = 4 # int(sol_per_pop /5) # int(np.ceil(sol_per_pop/3))# consider increasing this
    num_genes = len(power_list)

    parent_selection_type = "rank"
    keep_parents = 2 # int(np.ceil(sol_per_pop/5))
    
    mutation_type = "random"
    mutation_percent_genes = 10
    
    crossover_type = "single_point"
    # mutation_type = "adaptive"
    # mutation_percent_genes =  mutation_percent_genes = (20, 1) 
    
    possible_solutions = gene_upper_limit(power_list, demand) # gene space
    
    initial_pop  = initial_pop_reactors(power_list,  sol_per_pop, possible_solutions)
    
    allow_dup  = True
    
    


    
    def fitness_func(ga_instance, solution, solution_idx):
        # print("\n\n proposed solution", solution)
       
        power_list_modified = [power_list [i] for i in range(len(solution)) if solution[i] != 0]
      
        Num_of_each_reactor_type_modified = [x for x in solution if x != 0]
      
        long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)
       
        
        if sum(long_list_power) >= demand * capacity_factor_criteria and  sum(long_list_power) <= 2*demand : #  ((sum(long_list_power)) - min(long_list_power) ) <= demand:
            
               
                output_lcoe_results = level_cost_of_energy_reactor_mix_starting_from_BOAK_simplified(demand, interest_rate, solution, power_list, levelization_period_years, elec_price)
                output_lcoe  = output_lcoe_results[0]
                system_CF =  output_lcoe_results[1]

            
            
                fitness_1 = 1000 / ( output_lcoe) 
                
                if    3*capacity_factor_criteria >=system_CF >= capacity_factor_criteria:
                    fitness_2 = 0
                    
                else:
                    fitness_2 = -2
                
                fitness = fitness_1 + fitness_2
                
                GA_results =  { 'sol' :solution , 'CF' : np.round(system_CF, 3), 'LCOE' : np.round(output_lcoe, 3) }
                solutions_list.append(solution )
                CF_list.append(np.round(system_CF, 3))
                LCOE_list.append(np.round(output_lcoe, 3))
                
                with open("GA_results_simple_case.csv", "a") as csv_file:
                    writer = csv.writer(csv_file)
                    for key, value in GA_results.items():
                        writer.writerow([key, value])
         
                
                # print("solution is : " , solution)
                # print("Total Power is : " , sum(long_list_power))
                # print("MW discrepancy : " , np.abs(demand - sum(long_list_power)))
                # print("Capacity Factor (min): " , np.round(capacity_factor_t_min, 3))
                # print("LCOE : " , np.round(output_lcoe, 3))
                
        else:
            fitness = -2
        # if fitness >=0:    
        # print( "FITNESS NOW: ", fitness, "\n")
        
        

        return fitness
            
    ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating= num_parents_mating,
                       fitness_func= fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                     
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents, crossover_type=crossover_type,
                       
                    #    mutation_percent_genes= mutation_percent_genes,
                       
                       # pick 'possible_solutions' if you want to give more freedom,  or 'solutions_P_list' if you want to be faster.
                       gene_space=  possible_solutions ,
                       #mutation_type=mutation_type, 
                       stop_criteria= ["saturate_500"], 
                      
                       on_generation= on_gen,
                        fitness_batch_size=1,
                        keep_elitism = 0,
                        gene_type = int, initial_population = initial_pop,
                        parallel_processing = 8,
                    
                    
                       allow_duplicate_genes=allow_dup )

    
 
    ga_instance.run()
  
    end_time = time.time() 
    
    solution1, solution_fitness1, solution_idx1 = ga_instance.best_solution()
    print("Parameters of the best solution : {solution}".format(solution=solution1))
    print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness1))

    
    
    sol, sol_fitness, _ = ga_instance.best_solution()
    print("\n The optimization program runtime is " , np.round( (end_time -start_time), 0), " sec", " & The Number of Generations Passed is ",\
        ga_instance.generations_completed, "...... Fitness value of the best solution = {solution_fitness}".format(solution_fitness=sol_fitness))
    # print(sol, 'gg') 



    
    return solution1, solutions_list, CF_list, LCOE_list


In [6]:
power_list = [1000, 300 ,10] # we can decrease the number of solution if needed
levelization_period_years = 40
interest_rate = 0.06
elec_price = 0
best_solss = []
for Demand in [1310 , 1742, 2188, 2620]:
    for capacity_factor_criteria_1 in [0.95 , 0.96, 0.97, 0.98, 0.99, 1]: 
        print(f' Demand is {Demand} MWe &  Availability Criteria is {capacity_factor_criteria_1}' )
        optimization = optimize_lcoe( power_list ,  int(40*365/7), Demand, 0.06, capacity_factor_criteria_1, elec_price)  
        best_sol = optimization[0]
        best_solss.append(best_sol) # 1 1 0 ,    0 4 16

 Demand is 1310 MWe &  Availability Criteria is 0.95
Parameters of the best solution : [1 1 0]
Fitness value of the best solution = 10.853948176880234

 The optimization program runtime is  17.0  sec  & The Number of Generations Passed is  518 ...... Fitness value of the best solution = 10.853948176880234
 Demand is 1310 MWe &  Availability Criteria is 0.96
Parameters of the best solution : [ 0  4 11]
Fitness value of the best solution = 10.820204117605778

 The optimization program runtime is  20.0  sec  & The Number of Generations Passed is  649 ...... Fitness value of the best solution = 10.820204117605778
 Demand is 1310 MWe &  Availability Criteria is 0.97
Parameters of the best solution : [ 0  4 12]
Fitness value of the best solution = 10.793616717130803

 The optimization program runtime is  17.0  sec  & The Number of Generations Passed is  500 ...... Fitness value of the best solution = 10.793616717130803
 Demand is 1310 MWe &  Availability Criteria is 0.98
Parameters of the be

In [7]:
Demand_list = [1310 , 1742, 2188, 2620]
Longer_list = [item for item in Demand_list for _ in range(6)]
Longer_list_arr = np.array(Longer_list )
lcoess = []
CFSS = []
for i in range(len(best_solss)):
    ss = best_solss[i]
    finals = level_cost_of_energy_reactor_mix_starting_from_BOAK_simplified(Longer_list[i], interest_rate, ss, power_list, levelization_period_years, elec_price)
    lcoess.append(finals[0])
    CFSS.append(finals[1])
    

In [8]:
df = pd.DataFrame()
df['sol0']= np.vstack(best_solss)[:,0]
df['sol1']= np.vstack(best_solss)[:,1]
df['sol2']= np.vstack(best_solss)[:,2]

df['sol0 perent']= 100*pd.Series(np.divide(np.vstack(best_solss)[:, 0]* power_list[0], Longer_list_arr))
df['sol1 percent']= 100*pd.Series(np.divide(np.vstack(best_solss)[:, 1]* power_list[1], Longer_list_arr))
df['sol2 percent']= 100*pd.Series(np.divide(np.vstack(best_solss)[:, 2]* power_list[2], Longer_list_arr))
df['CF']=pd.Series(CFSS)
df['LCOE']=pd.Series(lcoess)


df.style.hide(axis="index") 

sol0,sol1,sol2,sol0 perent,sol1 percent,sol2 percent,CF,LCOE
1,1,0,76.335878,22.900763,0.000000,0.954198,91.689000
0,4,11,0.000000,91.603053,8.396947,0.962849,91.947950
0,4,12,0.000000,91.603053,9.160305,0.970308,92.175122
0,4,14,0.000000,91.603053,10.687023,0.985226,92.596140
0,4,15,0.000000,91.603053,11.450382,0.992685,92.791668
0,4,16,0.000000,91.603053,12.213740,1.000144,92.991573
0,6,0,0.000000,103.329506,0.000000,0.993553,84.161491
0,6,0,0.000000,103.329506,0.000000,0.993553,84.161491
0,6,0,0.000000,103.329506,0.000000,0.993553,84.161491
0,6,0,0.000000,103.329506,0.000000,0.993553,84.161491


In [11]:
def initial_pop_reactors(power_list, sol_per_pop, possible_solutions):
    """
    This function initializes a population of potential solutions for the reactor mix optimization problem.

    Parameters:
    - power_list (list): A list of possible reactor capacities.
    - sol_per_pop (int): The number of potential solutions per population.
    - possible_solutions (list): A list of upper limits for each reactor capacity.

    Returns:
    - pop (list): A list of potential solutions, where each sub-list represents a solution and contains integers representing the number of each reactor type.
    """
    print(sol_per_pop, len(power_list))
    pop = np.zeros((sol_per_pop, len(power_list)))
   
    for i in range(len(power_list)):
        temp = np.random.randint(0, max(possible_solutions[i]) + 1, size=sol_per_pop).tolist()
        pop[:, i] = temp
    return pop.tolist()







def gene_upper_limit(power_list, demand): # limiting the maximum number of each type of reactors

    upper_limit_list = []
 
    for i in range(len(power_list )):
        
        # I use the multiplier 1.05 because of the ratio between refueling duration and operational lifetime is will nnot be higher than 1.05
        upper_limit =range(int(np.ceil(1.05*demand/power_list[i]))+2) ## TODO check the +1
        upper_limit_list.append(upper_limit )

    return  upper_limit_list








   

# LCOE Sampling

In [12]:

def LCOE_similar_or_mixed(power_list, demand, potential_solution, interest_rate , levelization_period_weeks):
    if np.count_nonzero(potential_solution )  == 0:
        print("ERROR!!!")
        
    elif np.count_nonzero(potential_solution )  == 0:
        print("ERROR!!!")    
         
    elif np.count_nonzero(potential_solution ) >1: # mixed reactors:
        power_list_modified = [power_list [i] for i in range(len(potential_solution)) if potential_solution[i] != 0]
        Num_of_each_reactor_type_modified = [x for x in potential_solution if x != 0]


        long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)



        capacity_factor_results =   capacity_factor_weeks_approach_mix_reactors(long_list_power  ,levelization_period_weeks, demand)
        MWh_generated_per_year_per_demand_list = capacity_factor_results[4]
        MWh_excess_per_year_list =     capacity_factor_results[5]
        Tot_OM__cost_per_year_list =  capacity_factor_results[6]

        capacity_factor_t_min = min(capacity_factor_results[1])
        output_lcoe = level_cost_of_energy_reactor_mix_starting_from_BOAK( interest_rate, power_list_modified, Num_of_each_reactor_type_modified,\
                    MWh_generated_per_year_per_demand_list, MWh_excess_per_year_list, 0,\
                        Tot_OM__cost_per_year_list)
        return output_lcoe, capacity_factor_t_min


In [13]:
demand = 1500
potential_solution = [1, 5, 50]
power_list = [1000, 300, 10]
interest_rate = 0.06
levelization_period_weeks = int(40*365/7)
results = LCOE_similar_or_mixed(power_list, demand, potential_solution, interest_rate , levelization_period_weeks)

Schedule Generation :  1 [  0   3   6   9  12  15  19  22  25  28  31  35  38  41  44  47  50  54
  57  60  63  66  70  73  76  79  82  85  89  92  95  98 101 105 108 111
 114 117 120 124 127 130 133 136 140 143 146 149 152 155 159 162 165 168
 171 175] 0.003115264797507788
1 [[  0   3   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]
 ...
 [  0   3   6 ... 167 171 175]
 [  0   3   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]]
Schedule Generation :  2 [  0   3   6   9  12  15  19  22  25  28  31  34  38  41  44  47  50  54
  57  60  63  66  70  73  76  79  82  85  89  92  95  98 101 105 108 111
 113 117 120 124 127 129 133 136 140 143 146 149 152 155 159 162 165 168
 171 175] 0.003115264797507788
2 [[  0   3   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]
 [  0   3   6 ... 167 171 175]
 ...
 [  0   2   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]
 [  0   3   6 ... 168 171 175]]
Schedule Generation :  3 [  0   3   6   9  12  15  19  22  25 